### **TextLayer Data Guru: A Natural Language Querying Agent**

This outlines and demos "TextLayer Data Guru," a natural language querying agent using the powerful o3-mini CoT reasoning model for SQL generation. This agent is designed to facilitate speedy data interaction by converting natural language inputs into structured queries particular DuckDB SQL.

---

### **Design Considerations**

When developing an agent for information retrieval and summarization, several factors are typically considered:

*   **Information Accuracy:** Accuracy and precision and recall in information retrieval are important for reliable results.
*   **Speed:** The responsiveness of the agent contributes to user experience.
*   **Ease-of-Use:** Simplification of data access through natural language is a primary objective.
*   **Scalability:** The ability to handle varying loads and data volumes is a design consideration.

---

### **Application Architecture**

The "TextLayer Data Guru" application is implemented using the **Flask** framework. It utilizes a threaded executor for managing REST API calls. This approach is functional for a service that does not involve intensive CPU computation. It is noted that standard WSGI servers (e.g., Uvicorn, Gunicorn) are generally employed for horizontal scaling in production environments, which are not used in this specific setup.

---

### **Agent Design Overview**

The agent's core functionality relies on a carefully selected Language Model and a defined operational structure:

1.  **Router agent** The router agent was chosen to be gpt-4o and the system prompts for it were left untouched

2.  **Core Language Model for text-to-sql tool:** The agent's central component is a **o3-mini** based tool for text-to-sql generation, a fast Chain-of-Thought (CoT) reasoning model. This model has demonstrated effective performance in generating accurate SQL queries. As well as processing multi-step conversations and routing to tools. o3-mini genearlly outperforms gpt-4o-mini and gpt-4o on a variety of logical reasoning, code generation and complex instruction following tasks.

3.  **Tool Operating Modes:** The tool provides some extra modes for robust user input processing:
    *   **SQL Query Conversion:** Converts relevant natural language inputs into SQL queries.
    *   **Clarification Seeking:** Engages in clarifying questions when user input is ambiguous.
    *   **Irrelevant Input Refusal:** Identifies and declines inputs that are not relevant to database queries.

5.  **Database Context and SQL Syntax Context:** The tool is provided with a **complete database schema and example rows with non-null data entries**. This context aids in generating accurate SQL queries and allows for basic inquiries about database structure, such as tables and columns. Additonally a set of valid DuckDB/postgres SQL datetime commands were added to the system prompt to allow the agent ot use the correct SQL tools to process datetime operations and special instructions for ignoring null or onvalid fields in aggregate operations were added.

6.  **System Prompt Structure:** The LLM's system prompt is general yet includes specific instructions for handling ambiguous and irrelevant inputs. This design allows for flexibility and potential modification for incorporating additional tools.


---

### **RAG Augmentation for Few-Shot Example Injection**

To enhance the agent's ability to handle specific or niche user query patterns, a **Retrieval-Augmented Generation (RAG)** component is integrated into the tool:

1.  **Vector Store Index:** A simple llamaindex vector store index is used to store `key:value` pairs.
2.  **CSV Format for Examples:** The `key:value` pairs are stored in a CSV format, where `user_query` serves as the vectorized search key, and `sql_query` and `description` are the associated values.

    *   **Example 1:**
        ```csv
        user_query,sql_query,description
        tell me about Inez Santos,"(select * from customer where 'Inez Santos' in (Salesperson) limit 5) union all (select * from customer where 'Inez Santos' in (""Sales Manager"") limit 5)","Inez Santos appears to be the name of a person, print a summary of columns and rows limit 5 from tables that likely contain persons/names of people"
        ```
    *   **Example 2:**
        ```csv
        user_query,sql_query,description
        what products are there,select distinct name from product,"The user is asking broad about products, the best response is to generate a SQL query listing distinct products in the products table, assume that product.name contains the name of the products"
        ```

3.  **Few-Shot Example Injection:** The RAG system's purpose is to inject these few-shot examples into the agent's system prompt. This process is intended to improve the agent's performance for certain Business Intelligence (BI) and analytics-related user inputs.

---

### **Security Considerations**

Security measures are incorporated into the system's design:

1.  **Read-Only Database Connection:** The database connection is configured for read-only access. An explicit instruction preventing write operations is also included in the system prompt for the LLM.
2.  **Future Access Controls:** For production environments, the implementation of additional user-based access controls is identified as a potential area for further development.

---

### **Summary of File Modifications**

1. defined O3_MINI_MODEL in config since it is in the list of available models
2. /app/services/rag/ contains generate_rag_index.py which generates a vecor store index from the local CSV of few shot examples in /rag/data and rag_inference.py exposes a vvector inde retreiver for the agent to use
4. text_to_sql.py - modified tool to do o3-mini call and LLM results summary
5. /app/utils/duckdb_utils.py was created to host offline SQL and ETL operations that generate CSVs for DB schema and table data examples
6. /app/services/datastore/duckdb_datastore.py was modified to set read_only = True for the connector
7. requirements.txt - llama-index==0.13.0 was added
8. entire project was reformatted with black
9. various local CSV files were generated to give more database context to the agent

### **Test Cases**
1. A non-exhuastive set of basic chat based test cases are presented below

In [149]:
import requests
from time import time
import json
from IPython.display import Markdown, display

In [159]:
def query_textlayer_guru(url: str, user_content: str) -> dict | None:
    """
    Sends a natural language query to the TextLayer Data Guru API.

    Args:
        url (str): The URL of the TextLayer Data Guru API endpoint (e.g., "http://127.0.0.1:5000/v1/threads/chat").
        user_content (str): The natural language query string (e.g., "describe the available tables").

    Returns:
        dict | None: A dictionary containing the JSON response if successful, otherwise None.
    """
    payload = {
        "messages": [
            {
                "role": "user",
                "content": user_content
            }
        ]
    }

    try:
        start = time()
        response = requests.post(url, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        print("response took", time()-start, "s")
        return response.json()

    except requests.exceptions.HTTPError as errh:
        print(f"Http Error: {errh} - Response: {getattr(errh.response, 'text', 'No response text')}")
        return None
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
        return None
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
        return None
    except requests.exceptions.RequestException as err:
        print(f"An unexpected error occurred: {err}")
        return None
    except json.JSONDecodeError:
        print(f"Failed to decode JSON from response: {response.text}")
        return None

api_url = "http://127.0.0.1:5000/v1/threads/chat"

def get_and_print_response(query):
    print(f"--- Querying: {query} ---")
    response_data = query_textlayer_guru(api_url, query)

    if response_data and "payload" in response_data and isinstance(response_data["payload"], list):
        for item in response_data["payload"]:
            if "content" in item:
                if item["content"]:
                    display(Markdown(item["content"].lstrip("\"").rstrip("\"").replace("\\n", "\n")))
    elif response_data:
        print("Response JSON did not contain a valid 'payload' field.")
        print(json.dumps(response_data, indent=2))
    else:
        print(f"Failed to get a response for {query}")


In [211]:
# demo 1, asking a general question about the database
get_and_print_response("describe the available tables")

--- Querying: describe the available tables ---
response took 6.845021724700928 s


describe the available tables

| name             |
|:-----------------|
| account          |
| customer         |
| other            |
| product          |
| time             |
| time_perspective |
| version          |

__The available tables are: account, customer, other, product, time, time_perspective, and version.__

In [212]:
# demo 2, asking to show a summary about customers
get_and_print_response("show me 5 top rows from customers")

--- Querying: show me 5 top rows from customers ---
response took 8.941876649856567 s


show me 5 top rows from customers

| Key   | ParentId   | Name         | Channel   | Channel Parent   | Customer Since   | Industry   | Location   | Sales Manager   | Salesperson   |
|:------|:-----------|:-------------|:----------|:-----------------|:-----------------|:-----------|:-----------|:----------------|:--------------|
| C0000 |            | Total Region |           |                  |                  |            |            |                 |               |
| C1000 | C0000      | West         |           |                  |                  |            |            |                 |               |
| C2000 | C0000      | Midwest      |           |                  |                  |            |            |                 |               |
| C3000 | C0000      | Southeast    |           |                  |                  |            |            |                 |               |
| C4000 | C0000      | Southwest    |           |                  |                  |            |            |                 |               |

__The first five entries in the customers table display the hierarchical structure of regions, with \"Total Region\" at the top and \"West,\" \"Midwest,\" \"Southeast,\" and \"Southwest\" as sub-regions. No additional customer-specific data is provided in the output.__

In [213]:
get_and_print_response("which salesperson had the most customer accounts")

--- Querying: which salesperson had the most customer accounts ---
response took 8.461036443710327 s


which salesperson had the most customer accounts

| Salesperson    |   account_count |
|:---------------|----------------:|
| Clarita Kiefer |            1398 |

__The salesperson with the most customer accounts is Clarita Kiefer, with 1398 accounts.__

In [214]:
# a general query concerning a salesperson or possible a sales manager
get_and_print_response("tell me about Christa Peters")

--- Querying: tell me about Christa Peters ---
response took 16.882885932922363 s


tell me about Christa Peters

| Key     | ParentId   | Name                         | Channel       | Channel Parent   | Customer Since   | Industry     | Location   | Sales Manager   | Salesperson    |
|:--------|:-----------|:-----------------------------|:--------------|:-----------------|:-----------------|:-------------|:-----------|:----------------|:---------------|
| C000018 | C5000      | Cleantech North              | Wholesale     | Total Channel    | 5/25/2017        | Technology   | CA-ON      | Jim Halpert     | Christa Peters |
| C000033 | C1000      | Pioneering Planning Holdings | Wholesale     | Total Channel    | 3/16/2014        | Construction | US-FL      | Saul Goodman    | Christa Peters |
| C000040 | C1000      | Stellar Frameworks Network   | Distributor   | Total Channel    | 9/7/2015         | Technology   | GB         | Jim Halpert     | Christa Peters |
| C000044 | C1000      | Profound Tech Development    | Private Label | Total Channel    | 10/28/2011       | Retail       | US-NY      | Don Draper      | Christa Peters |
| C000115 | C1000      | Optimized Labs Corporation   | Retail        | Total Channel    | 6/22/2018        | Retail       | DE         | Don Draper      | Christa Peters |

__Christa Peters is associated with multiple companies across various industries such as Technology, Construction, and Retail. She works with sales managers Jim Halpert, Saul Goodman, and Don Draper, and is associated with channels like Wholesale, Distributor, Private Label, and Retail.__

In [215]:
get_and_print_response("who had more customer accounts Christa Peters or Saul Goodman")

--- Querying: who had more customer accounts Christa Peters or Saul Goodman ---
response took 22.127484798431396 s


who had more customer accounts Christa Peters or Saul Goodman

|   count_star() |
|---------------:|
|        1336.00 |

__Christa Peters holds 1,336 customer accounts.__

|   count_star() |
|---------------:|
|        6500.00 |

__Saul Goodman holds 6,500 customer accounts.__

In [216]:
get_and_print_response("how many customer accounts did Saul Goodman have as a sales manager")

--- Querying: how many customer accounts did Saul Goodman have as a sales manager ---
response took 7.32975435256958 s


how many customer accounts did Saul Goodman have as a sales manager

|   customer_count |
|-----------------:|
|          6500.00 |

__Saul Goodman had 6,500 customer accounts as a sales manager.__

In [229]:
# a somewhat ambiguous query as Saul is a sales manager and a crooked lawyer
get_and_print_response("how many customer accounts did Saul Goodman have as a salesman")

--- Querying: how many customer accounts did Saul Goodman have as a salesman ---
response took 13.83296012878418 s


how many customer accounts did Saul Goodman have as a salesman

|   account_count |
|----------------:|
|         6500.00 |

__Saul Goodman had 6,500 customer accounts as a salesman.__

In [230]:
# a the above query with a little more clarification
get_and_print_response("how many customer accounts did Saul Goodman have as a salesman, sales managers do not sell, only manage")

--- Querying: how many customer accounts did Saul Goodman have as a salesman, sales managers do not sell, only manage ---
response took 11.582584381103516 s


how many customer accounts did Saul Goodman have as a salesman, sales managers do not sell, only manage

|   count_star() |
|---------------:|
|           0.00 |

__Saul Goodman did not have any customer accounts as a salesman. Sales managers only manage and do not sell.__

In [231]:
# this query requires the agent to know how to parse out year from the year field
get_and_print_response("in what year were the most customers signed on")

--- Querying: in what year were the most customers signed on ---
response took 11.175101280212402 s


in what year were the most customers signed on

|   signup_year |   num_signups |
|--------------:|--------------:|
|          2011 |          1694 |

__The year with the most new customer sign-ups was 2011, with 1,694 sign-ups.__

In [220]:
get_and_print_response("give counts of customer signed on per year list by largest")

--- Querying: give counts of customer signed on per year list by largest ---
response took 9.416120052337646 s


give counts of customer signed on per year list by largest

|   signup_year |   customer_count |
|--------------:|-----------------:|
|       2011.00 |          1694.00 |
|       2012.00 |          1673.00 |
|       2015.00 |          1673.00 |
|       2019.00 |          1663.00 |
|       2013.00 |          1648.00 |
|       2021.00 |          1647.00 |
|       2014.00 |          1644.00 |
|       2022.00 |          1634.00 |
|       2017.00 |          1632.00 |
|       2016.00 |          1630.00 |
|       2010.00 |          1628.00 |
|       2009.00 |          1625.00 |
|       2018.00 |          1606.00 |
|       2023.00 |          1587.00 |
|       2008.00 |          1559.00 |
|       2020.00 |          1545.00 |

__In the year 2011, the highest number of customers signed up, totaling 1694. The list is ordered by the largest customer count per year.__

In [233]:
# need to tell tool to ignore rows with NA, NULL or '' cells in aggregate fields
get_and_print_response("give counts of customer signed on per year list by earliest")

--- Querying: give counts of customer signed on per year list by earliest ---
response took 12.037032127380371 s


give counts of customer signed on per year list by earliest

|   sign_year |   customer_count |
|------------:|-----------------:|
|     2008.00 |          1559.00 |
|     2009.00 |          1625.00 |
|     2010.00 |          1628.00 |
|     2011.00 |          1694.00 |
|     2012.00 |          1673.00 |
|     2013.00 |          1648.00 |
|     2014.00 |          1644.00 |
|     2015.00 |          1673.00 |
|     2016.00 |          1630.00 |
|     2017.00 |          1632.00 |
|     2018.00 |          1606.00 |
|     2019.00 |          1663.00 |
|     2020.00 |          1545.00 |
|     2021.00 |          1647.00 |
|     2022.00 |          1634.00 |
|     2023.00 |          1587.00 |

__The table provides customer counts signed per year from 2008 to 2023, sorted by the earliest year first. The highest customer count was in 2011 with 1,694 customers, and the lowest was in 2020 with 1,545 customers.__

In [234]:
get_and_print_response("tell me about what products there are, limit to 20 products")

--- Querying: tell me about what products there are, limit to 20 products ---
response took 9.673447608947754 s


tell me about what products there are, limit to 20 products

| Name        |
|:------------|
| Product 62  |
| Product 64  |
| Product 110 |
| Product 127 |
| Product 136 |
| Product 153 |
| Product 156 |
| Product 161 |
| Product 173 |
| Product 178 |
| Product 218 |
| Product 242 |
| Product 278 |
| Product 301 |
| Product 308 |
| Product 310 |
| Product 316 |
| Product 330 |
| Product 334 |
| Product 336 |

__The products available are Product 62, Product 64, Product 110, Product 127, and so on, up to Product 336. The list includes a total of 20 products.__

In [235]:
get_and_print_response("whats the weather like in Vancouver")

--- Querying: whats the weather like in Vancouver ---
response took 2.1283481121063232 s


whats the weather like in Vancouver

I'm unable to access real-time data, including current weather conditions. However, you can easily check the weather in Vancouver by using a weather website or app such as Weather.com, AccuWeather, or a local news station's website.

In [236]:
# a query joining two different tables
# generated query is SELECT v.Key, v.StartPeriod, t.Name AS MonthName FROM main.version v JOIN main.time t ON SUBSTR(v.StartPeriod, -3) = SUBSTR(t.Month, -3) ORDER BY v.StartPeriod DESC LIMIT 5;

In [237]:
get_and_print_response("list the most recent versions and their monthly start period cross reference month in main.time limit 5")

--- Querying: list the most recent versions and their monthly start period cross reference month in main.time limit 5 ---
response took 14.94752836227417 s


list the most recent versions and their monthly start period cross reference month in main.time limit 5

| Key   | Name         | StartPeriod   | Month_Name   |
|:------|:-------------|:--------------|:-------------|
| Q3FOR | 9+3 Forecast | 2025M10       | October      |
| Q3FOR | 9+3 Forecast | 2025M10       | October      |
| Q3FOR | 9+3 Forecast | 2025M10       | October      |
| Q3FOR | 9+3 Forecast | 2025M10       | October      |
| Q3FOR | 9+3 Forecast | 2025M10       | October      |

__The most recent version is \"9+3 Forecast\" with a monthly start period of \"2025M10,\" corresponding to October. The results include 5 entries of this version and start period.__

### multi turn conversation example

In [238]:
      
import requests
import json
from IPython.display import Markdown, display

api_url = "http://127.0.0.1:5000/v1/threads/chat"

def query_textlayer_guru_multiturn(url: str, messages: list) -> dict | None:
    payload = {
        "messages": messages
    }

    try:
        start = time()
        response = requests.post(url, json=payload)
        response.raise_for_status()
        print("response took", time()-start, "s")

        return response.json()

    except requests.exceptions.HTTPError as errh:
        print(f"Http Error: {errh} - Response: {getattr(errh.response, 'text', 'No response text')}")
        return None
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
        return None
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
        return None
    except requests.exceptions.RequestException as err:
        print(f"An unexpected error occurred: {err}")
        return None
    except json.JSONDecodeError:
        response_text = getattr(response, 'text', 'No response text available') if 'response' in locals() else 'No response object'
        print(f"Failed to decode JSON from response: {response_text}")
        return None

def get_multiturn_response(current_query: str, history: list = None) -> list:
    if history is None:
        history = []

    user_message = {"role": "user", "content": current_query}
    messages_to_send = history + [user_message]

    print(f"--- User Query: {current_query} ---")
    response_data = query_textlayer_guru_multiturn(api_url, messages_to_send)

    new_history = []
    if response_data and "payload" in response_data and isinstance(response_data["payload"], list):
        full_conversation_payload = response_data["payload"]

        # Filter the full payload to create the new_history for the next turn
        for message in full_conversation_payload:
            if message["content"]:
                # quick fix to make sure tool role is chnaged to assistant role
                if  message["role"] == "tool":
                     message["role"] = "assistant"
                new_history.append({"role": message["role"], "content": message["content"]})
            # Handle tool messages for display, but don't add them to the conversational history for next turn

        print("\n--- Assistant's Response ---")
        # Display only the new messages generated in this turn
        # This assumes the payload returns the entire thread, including the sent messages
        new_messages_start_index = len(messages_to_send)
        for message in full_conversation_payload[new_messages_start_index:]:
            if message["role"] == "assistant":
                if message.get("content") is not None:
                    display(Markdown(message["content"].lstrip("\"").rstrip("\"").replace("\\n", "\n")))
                if message.get("tool_calls"):
                    print(f"Assistant invoked tools: {json.dumps(message['tool_calls'], indent=2)}")
            elif message["role"] == "tool":
                if message.get("content") is not None:
                    print(f"--- Tool Output ({message.get('tool_call_id', 'N/A')}): ---")
                    display(Markdown(message["content"].lstrip("\"").rstrip("\"").replace("\\n", "\n")))
        
        return new_history
    elif response_data:
        print("Response JSON did not contain a valid 'payload' field.")
        print(json.dumps(response_data, indent=2))
        return history # Return original history if response is malformed
    else:
        print(f"Failed to get a response for {current_query}")
        return history # Return original history on failure

    

In [239]:
hist = get_multiturn_response("how many customers did Saul Goodman have")

--- User Query: how many customers did Saul Goodman have ---
response took 9.198768377304077 s

--- Assistant's Response ---
Assistant invoked tools: [
  {
    "function": {
      "arguments": "{\"user_query\":\"how many customers did Saul Goodman have\"}",
      "name": "text_to_sql_tool"
    },
    "id": "call_HxNQ6xHhBKI7XGWOAZOoeuWR",
    "type": "function"
  }
]


|   customer_count |
|-----------------:|
|          6500.00 |

__Saul Goodman had 6,500 customers.__

In [240]:
print(hist)

[{'role': 'user', 'content': 'how many customers did Saul Goodman have'}, {'role': 'assistant', 'content': '"|   customer_count |\\n|-----------------:|\\n|          6500.00 |\\n\\n__Saul Goodman had 6,500 customers.__"'}]


In [241]:
hist = get_multiturn_response("how many customers did Christa Peters have", hist)

--- User Query: how many customers did Christa Peters have ---
response took 9.382657766342163 s

--- Assistant's Response ---
Assistant invoked tools: [
  {
    "function": {
      "arguments": "{\"user_query\":\"how many customers did Christa Peters have\"}",
      "name": "text_to_sql_tool"
    },
    "id": "call_csCFSD3GCKMCqdNIdfdYszuq",
    "type": "function"
  }
]


|   num_customers |
|----------------:|
|         1336.00 |

__Christa Peters had 1,336 customers.__

In [242]:
hist = get_multiturn_response("from the previous 2 results who had the most customers", hist)

--- User Query: from the previous 2 results who had the most customers ---
response took 1.5285320281982422 s

--- Assistant's Response ---


Saul Goodman had the most customers, with a total of 6,500 customers, compared to Christa Peters who had 1,336 customers.

In [245]:
# ask ambiguous query, wait for clarifying question
hist = get_multiturn_response("how did Saul Goodman have")

--- User Query: how did Saul Goodman have ---
response took 2.006071090698242 s

--- Assistant's Response ---


Could you please clarify what specific information or aspect you are looking to know about Saul Goodman? Are you referring to his character from "Breaking Bad" and "Better Call Saul," his professional strategies, his legal practices, or something else entirely?

In [246]:
# provide resolution to above
hist = get_multiturn_response("how many customers did he have", hist)

--- User Query: how many customers did he have ---
response took 6.985926389694214 s

--- Assistant's Response ---
Assistant invoked tools: [
  {
    "function": {
      "arguments": "{\"user_query\":\"How many customers did Saul Goodman have?\"}",
      "name": "text_to_sql_tool"
    },
    "id": "call_pq1UJwuuX5oQ51zuq4M5SZ6D",
    "type": "function"
  }
]


|   count_star() |
|---------------:|
|        6500.00 |

__Saul Goodman had 6,500 customers.__

In [243]:
# and this passes the multi turn conversation test

### **Discussion**
1. The test suite above demonstrates basic capability with some queries that canbe expected for a simple text-to-sql chat service for BI/analytics. All test cases pass but a more thourough testing suite is needed.
2. The agent incoporates data augmentation a two levels, first is database schema and best attempt non-null row example from each table, second is a RAG retriever for few shot text-to-sql examples
3. More advanced implementations will need to review business use cases, requirements and analyzed for latency, safety and scalability
4. Due to time constraints the functional test coverage is not exhuastive. Functional and end-to-end testing will require large amounts (at least 200+, 1000 - 10000+ is ideal) of ground truth data and automated test suites to compute accuracy curves, and potentially multiple programs of feature testing such as switching out for different LLMs are needed
5. More advanced integrations with multiple data sources such as RDS and RAG systems can be considered to augment the capability of the system, eg. multi database querying, retrival of information from sources other than a SQL database etc.

### This has been a fun exercise and I am happy to discuss the results!